# Biocomputing with Brainoware
### Procedure 3 - Brainoware software framework
#### 3. Information decoding
**Author: Hongwei Cai, Huiyu Chu**  
**Date: June 6, 2025**  
**Description**: This part describes:
1) how to retrieve spike properties and stimulation timing from the raw recording files.
2) mask out spikes during stimulation artifact periods.
3) select top N active electrodes
4) count spikes of active electrodes, merge spike counts together as the organoid's reservoir states

In [1]:
import h5py
import numpy as np
import pandas as pd
import os
h5py.enable_ipython_completer()

path_to_rawfile = "./2_Electrical_stimulation_and_recording_data"
raw_files = [f for f in os.listdir(path_to_rawfile) if f.endswith(".raw.h5")]
raw_files

['speech_recognition_10hz_75mv_21680.raw.h5']

In [2]:
for file in raw_files:
    # -- load the raw recording file into a h5 file object -- #
    h5_file_object = h5py.File(os.path.join(path_to_rawfile, file),'r')
    # -- get spike information containing frame numbers, channels, and amplitude. Convert frame numbers into time in seconds -- #
    frame_numbers = h5_file_object['data_store']['data0000']['spikes']['frameno']
    channels = h5_file_object['data_store']['data0000']['spikes']['channel']
    amplitudes = h5_file_object['data_store']['data0000']['spikes']['amplitude']
    # convert integer channels to strings and append ch_ prefix to them 
    channels_as_string = channels.astype(str)
    channels_as_string = np.char.add("ch_",channels_as_string)
    # get the initial frame number corresponding to the recording beginning
    frame_number_start = np.min(frame_numbers)
    # transform successive frame numbers into time in seconds relative to the beginning frame, by calculating the frame difference divided by the sampling rate (20k Hz)
    time_after_start_frame=(frame_numbers-frame_number_start)/20000
    
    # -- similarly, get the stimulation event framenumbers and convert them into time in seconds relative to the beginning frame -- #
    stimulation_timepoints = []
    for event in h5_file_object['data_store']['data0000']['events'][:]:
        stimulation_timepoints.append((event[0]-frame_number_start)/20000)
    
    # -- save spike information and stimulation timing information to separate csv files -- #
    spikes=np.stack(
        (time_after_start_frame,
         channels_as_string,
         amplitudes),
        axis=-1)
    df_spikes = pd.DataFrame(spikes, columns = ['time','channel','amplitude'])
    df_sti = pd.DataFrame(stimulation_timepoints, columns = ['time'])
    save_folder = "./3_Information_decoding/spike_stimulation_file"
    spike_filename = file.replace('.raw.h5', '_spike.csv')
    stimulation_time_filename = file.replace('.raw.h5', '_sti_time.csv')
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    df_spikes.to_csv(os.path.join(save_folder, spike_filename))
    df_sti.to_csv(os.path.join(save_folder, stimulation_time_filename))

#### PAUSE POINT 

The codes above concludes the retrieval of spike properties and stimulation timing from the raw recording files. The two saved csv files will be further processed to extract features for the training and prediction of the next multinomial linear regression layer.

In [3]:
import pandas as pd
import numpy as np
import os
import pickle 

spike_stimulation_path= "./3_Information_decoding/spike_stimulation_file/"
base_file_names = [f for f in os.listdir(spike_stimulation_path) if f.endswith("_spike.csv")]
# one can do batch process here, but only one example is used now to demonstrate the protocol.
example_base_file = base_file_names[0].replace("_spike.csv","")

spike_dataframe=pd.read_csv(spike_stimulation_path+example_base_file+'_spike.csv',usecols=[1,2,3],dtype="string")
spike_dataframe = spike_dataframe.astype({"time":"float"})

stim_dataframe = pd.read_csv(spike_stimulation_path + example_base_file + "_sti_time.csv",usecols=[1] )
stim_times = stim_dataframe.iloc[:, 0].values
stim_times_str = stim_times.astype(str)
stim_start_time = stim_times[0]

In [4]:

## -- mask out spikes that are within [-0.0008,0.002]s of stimulation times, since these spikes can be artifacts -- ##

# find the index range for each stimulation time
idx_start = np.searchsorted(spike_dataframe['time'].values, stim_times - 0.0008)
idx_end = np.searchsorted(spike_dataframe['time'].values, stim_times + 0.002)

# create a boolean mask (all initially True)
mask = np.ones(len(spike_dataframe), dtype=bool)

# set False for indices within artifact times
for start, end in zip(idx_start, idx_end):
    mask[start:end] = False
    
# Apply mask to filter DataFrames
spike_dataframe_no_artifact = spike_dataframe[mask]
print(spike_dataframe.shape)
print(spike_dataframe_no_artifact.shape)

(6322520, 3)
(1567331, 3)


In [ ]:
## select top 64 active electrode by descendingly sort the spike counts of all electrodes 
top_n=64
active_channels = spike_dataframe_no_artifact['channel'].value_counts()[:top_n].index.tolist()
spike_dataframe_no_artifact=spike_dataframe_no_artifact.astype({'time': 'str'})
# retrieve all spikes of active electrodes
spike_dataframe_no_artifact_active=spike_dataframe_no_artifact[spike_dataframe_no_artifact.apply(lambda x: x.isin(active_channels)).any(axis=1)]
spike_dataframe_no_artifact_active=spike_dataframe_no_artifact_active.astype({'time': 'float'})
# re-organize spike information as a list per active electrode (channel)
spike_dataframe_per_active_channel = spike_dataframe_no_artifact_active.groupby('channel',as_index=False).agg(lambda x: x.tolist())
spike_ndarray_per_active_channel=spike_dataframe_per_active_channel['time'].to_numpy()

In [6]:
def count_binned_spikes(spike_times,dt,count_start,count_end):
    """
    Function that count spikes within time bins
    Parameters
    ----------
    spike_times: an array of arrays
        an array of electrode channels. within each electrode's array is an array containing all the spike times of that electrode channel
    dt: number (any format)
        size of time bins
    count_start: number (any format)
        the start time of spike counting
    count_end: number (any format)
        the end time of spike counting
    Returns
    -------
    count_result : a matrix of size "number of time bins" x "number of channels"
        the number of spikes in each time bin for each channel
    """
    edges=np.arange(count_start,count_end,dt) #Get edges of time bins
    num_bins=edges.shape[0]-1 #Number of bins
    num_channels=spike_times.shape[0] #Number of neurons
    count_result=np.empty([num_bins,num_channels]) #Initialize array for binned neural data
    #Count number of spikes in each bin for each neuron, and put in array
    for i in range(num_channels):
        count_result[:,i]=np.histogram(np.array(spike_times[i]),edges)[0]
    return count_result

In [ ]:

## time-related parameters in the stimulation and recording section ##
## modify if using your own datasets ##
#   seconds_each_audio: For each audio, there are 5 seconds stimulation and recording time.
#   audios_in_each_classes: For each class (speaker), there are 30 audios to input.
#   classes: All classes in a classification task, in our case there are eight speakers.
#   dt: time window size of spike count
#   count_start_time: reference time point to start count spikes
#   count_end_time: reference time point to end count spikes
##
seconds_each_audio=5
audios_in_each_classes = 30
classes = 8
dt=0.01
count_start_time = 0.006
count_end_time = 4.105
## define stim_start_time_per_category - an 8 timepoint list as stimulation starting times of eight classes.
stim_start_time_per_category=np.arange(0,seconds_each_audio*audios_in_each_classes*classes,seconds_each_audio*30)
stim_start_time_per_category = stim_start_time+stim_start_time_per_category
## define stim_start_time_per_audioclip - an 30 timepoint list as stimulation starting time of 30 audioclips per class.
stim_start_time_per_audioclip = np.arange(0, seconds_each_audio*audios_in_each_classes, seconds_each_audio)

## store spike count data into spike_count_data list
spike_count_data=list()

## store reservoir states of all audioclips into spike_count_data
for time_category in stim_start_time_per_category:
    for time_audioclip in stim_start_time_per_audioclip:
        wdw_start=time_category + time_audioclip + count_start_time
        wdw_end=wdw_start + count_end_time
        count_result =count_binned_spikes(spike_ndarray_per_active_channel,dt,wdw_start,wdw_end)
        count_result_flattened = count_result.flatten()
        spike_count_data.append(count_result_flattened)

## save as pickle file for future use
save_path = "./3_Information_decoding"
os.makedirs(save_path, exist_ok=True)
with open(f'{save_path}/spike_count_data.pkl', 'wb') as f:
    pickle.dump(spike_count_data, f)